# Air Delhi Dataset Reproducibility Study

The purpose of this notebook is to assess the feasibility of developing further models on this dataset


**Config**

In [1]:
DATA_FOLDER = '/kaggle/input/airdelhi-dataset/PM Datasets/'
datadir = DATA_FOLDER

## Preprocessing

In [2]:
import tqdm
import torch
import argparse
import math
import pickle as pkl
import pandas as pd
import numpy as np
from haversine import haversine, haversine_vector, Unit
from sklearn.model_selection import KFold

mytqdm = tqdm.notebook.tqdm if 0 else tqdm.tqdm

class Args:
    num_sensors = 60
    train_start_day = 12
    date=None
    mfile=None
    datadir=DATA_FOLDER
    canada=0
    usa=0
    spatiotemporal=1
args = Args()

if args.usa:
    args.canada = 3

canada = args.canada
rand_sensor_locs = 1

fold = [3,5][rand_sensor_locs]

heatmap = 0
datadir = args.datadir
pm = ['pm10','pm2_5'][1]
pm2 = ['pm10','pm2_5'][0]
sensorTyp = ['random'][0]

km = 1
grouptime = 30

if canada == 1:
    nTotalDays = nTrainStartDay = 15
    args.num_sensors = 10
elif canada == 2:
    nTrainStartDay = 6
    nTotalDays = 16
elif canada == 3: # USA
    nTrainStartDay = 1
    nTotalDays = 10
else:
    nTrainStartDay = args.train_start_day
    nTotalDays = 91

nTestStartDay = nTrainStartDay           # The first test day, overridden by 'next'
skipdays = None
latlongprecise = 4
metricprecise = 3
if canada:
    ll_min, ll_max = [0, 43.135, -80.025], [0, 43.35, -79.638]
else:
    ll_min, ll_max = [11.0, 28.48, 77.1], [45.0, 28.72, 77.33]
ll_off = [1, np.round(0.00902*km,latlongprecise), np.round(0.01017*km,latlongprecise)]
g_lim = 0

date = 'all,next'
if args.date is not None:
    date = args.date

###############################################################################

def to_dt(time_string):
    return pd.to_datetime(time_string).tz_localize('UTC').tz_convert('Asia/Kolkata')

def preprocess_data(date, data_dir = "./data/", hour = None):
    file = date + "{}.csv".format('_all' if canada == 0 else '')
    print('Reading', file)
    df = pd.read_csv(data_dir + file, index_col = 0, parse_dates = ["dateTime"])
    # filter time
    if not canada:
        df = df[(df.dateTime >= to_dt(date)) & (df.dateTime <= to_dt(date+ " 18:00:00"))].reset_index(drop = True)

    # day subset
    dfHour = df[['dateTime','lat','long',pm,pm2]]

    # hour subset
    if hour:
        dfHour["hour"] = dfHour.dateTime.dt.hour
        dfHour = dfHour[dfHour.hour.isin(hour)]
        dfHour = dfHour.drop("hour", axis = 1)

    # preprocessing time
    dfHour.dateTime = dfHour.dateTime.dt.round('{}min'.format(grouptime))
    dfHour.dateTime = pd.to_datetime(dfHour.dateTime)
    if canada != 1:
        dfHour.dateTime = dfHour.dateTime.dt.hour*60+dfHour.dateTime.dt.minute
        dfHour.dateTime %= 1440
        dfHour = dfHour[(dfHour.dateTime>=300) & (dfHour.dateTime<=1350)]
    dfHour = dfHour.sort_values(by = ['dateTime','lat','long'])
    dfHour = dfHour.reset_index(drop = True)

    def round_val(val, min, off):
        val1 = ((val - min) / off).astype(int)
        val2 = (val1 * off) + min
        return round(val2, latlongprecise)
    dfHour.lat = round_val(dfHour.lat.astype(float), ll_min[1], ll_off[1])
    dfHour.long = round_val(dfHour.long.astype(float), ll_min[2], ll_off[2])

    # meaning pm values
    df = dfHour.groupby(['dateTime','lat','long']).mean().reset_index()
    df.loc[:, pm] = df.loc[:, pm].round(2)
    df.loc[:, pm2] = df.loc[:, pm2].round(2)

    return df.values

def find_best_sensor_loc(data, lim=0):
    loc = dict()
    for i,d in enumerate(data):
        if d[1] not in loc.keys():
            loc[d[1]] = dict()
        if d[2] not in loc[d[1]].keys():
            loc[d[1]][d[2]] = []
        loc[d[1]][d[2]].append(i)
    L = []
    for d1 in loc.keys():
        for d2 in loc[d1].keys():
            L.append([d1, d2, len(loc[d1][d2])])
    L = np.array(L)
    Lall = L.copy()
    if sensorTyp.startswith('rand'):
        L = L[L[:, 2] > lim]
        if len(L) > args.num_sensors:
            idx = np.random.choice(len(L), args.num_sensors)
            LL = L[idx, :2]
        else:
            LL = L[:, :2]
    else:
        LL = L[L[:, 2].argsort()][::-1]
        LL = LL[:args.num_sensors,:2]
    return LL, Lall

def num_sensor_loc(sensors, data):
    loc = dict()
    for i,d in enumerate(data):
        if d[1] not in loc.keys():
            loc[d[1]] = dict()
        if d[2] not in loc[d[1]].keys():
            loc[d[1]][d[2]] = []
        loc[d[1]][d[2]].append(i)
    L = []
    for d1 in loc.keys():
        for d2 in loc[d1].keys():
            L.append([d1, d2, len(loc[d1][d2])])
    L = np.array(L)
    lim, cnt = 0, 0
    L = L[L[:, 2] > lim]
    for s in sensors:
        for LL in L:
            if s[0] == LL[0] and s[1] == LL[1]:
                cnt += 1
                break
    return cnt

def save(data, file):
    pkl.dump(data, open(file,"wb"))

#####################################################################################

def xform_day(day):
    if canada:
        return 'canada_20{:02d}'.format(day)
    arr = [0, 30, 61]
    w = 0 if day <= 30 else 1 if day <= 61 else 2
    mon = ['2020-11-', '2020-12-', '2021-01-'][w]
    date = mon + '{:02d}'.format(day - arr[w])
    return date

#####################################################################################

cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')
print('Cuda:', cuda, '| Device:', device)
print('Num Sensors:', args.num_sensors)
print('Policy:', sensorTyp)

dates_in = date.split(',')
dates_train = []

# Preprocess
wy = 3
all_data = dict()

for day in range(min(nTrainStartDay,nTestStartDay), 1+nTotalDays):
    if skipdays is None or day not in skipdays:
        d = xform_day(day)
        dates_train.append(d)
        all_data[d] = preprocess_data(d, DATA_FOLDER)

if canada == 1:
    print('For 2015 data')
    dates_train = ['2015-02-12', '2015-02-13', '2015-03-19', '2015-03-25', '2015-03-27', '2015-04-01', '2015-04-10', '2015-04-15',
     '2015-04-17', '2015-04-21', '2015-04-24', '2015-05-12',  '2015-05-14', '2015-05-20', '2015-05-22', '2015-09-10', '2015-09-23', '2015-11-23']

if args.usa:
    df = pd.read_csv('city_pollution_data.csv')
    df = df[['Date', 'Time', 'latitude', 'longitude', 'pm25_median', 'pm10_median']]
    df.columns = ['Date', 'Time', 'lat', 'long', 'pm2_5', 'pm10']
    ll_min, ll_max = [0, 21.3, -157.9], [0, 47.6, -71.0]
    not_nan = [not math.isnan(i) for i in df.pm2_5.values]
    df = df[not_nan]

    def round_val(val, min, off):
        val1 = ((val - min) / off).astype(int)
        val2 = (val1 * off) + min
        return round(val2, latlongprecise)
    df.lat = round_val(df.lat.astype(float), ll_min[1], ll_off[1])
    df.long = round_val(df.long.astype(float), ll_min[2], ll_off[2])

    all_data = dict()
    dates_train = df.Date.unique().tolist()
    for dt in dates_train:
        D = df[df.Date == dt]
        all_data[dt] = D[['Time', 'lat', 'long', 'pm2_5', 'pm10']].values
if 1:
    # Gather All Data, and have independent sensors
    testData = np.zeros((0, 5))
    gloc = dict()
    for dateIdx, dt in enumerate(dates_train):
        testData = np.concatenate((testData, all_data[dt]), axis=0)
        loc = dict()
        for i, d in enumerate(all_data[dt]):
            if d[1] not in loc.keys():
                loc[d[1]] = dict()
            if d[2] not in loc[d[1]].keys():
                loc[d[1]][d[2]] = 1
                if d[1] not in gloc.keys():
                    gloc[d[1]] = dict()
                if d[2] not in gloc[d[1]].keys():
                    gloc[d[1]][d[2]] = 0
                gloc[d[1]][d[2]] += 1
            else:
                loc[d[1]][d[2]] += 1

    print('Data Shape', testData.shape)
    sensors, allsensors = find_best_sensor_loc(testData, lim=g_lim)

def get_spatial_locs(X):
    dstr = ["{}_{}".format(x[1], x[2]) for x in X]
    L = list(set(dstr))
    return np.array(L)

S = dict()
seen, unseen = 0, 1
for dateIdx, dt in enumerate(dates_train):
    dt2 = dt
    if args.usa:
        dt2 = dt.split('/')
        dt2 = '{}-{:02d}-{:02d}'.format(dt2[2], int(dt2[0]), int(dt2[1]))
        print('\'{}\','.format(dt2))
    S[dt2] = []
    data = []

    kf = KFold(n_splits=fold, shuffle=True, random_state=0)
    X = all_data[dt]
    if args.spatiotemporal:
        for train, test in kf.split(X):
            data.append([X[train], X[test]])
            S[dt2].append(X[train][:, :3])
    else:
        L = get_spatial_locs(X)
        for trainL, testL in kf.split(L):
            trainLL = L[trainL]
            testLL = L[testL]
            train, test = [],[]
            for i, x in enumerate(X):
                k = "{}_{}".format(x[1], x[2])
                if k in trainLL:
                    train.append(i)
                elif k in testLL:
                    test.append(i)
                else:
                    raise 'Error'
            data.append([X[train],X[test]])
            S[dt2].append(X[train][:,1:3])

    output_datadir = "/kaggle/working/"

    for f in range(fold):
        if rand_sensor_locs == 0:
            data.append([[], []])
            for d in all_data[dt]:
                dd = '{}_{}'.format(d[1], d[2])
                w = seen if dd in SS[f] else unseen
                data[f][w].append(d)
        header = ['dateTime', 'lat', 'long', 'pm2_5', 'pm10']
        df = pd.DataFrame(data[f][seen])
        df.columns = header
        df.to_csv('{}/{}_f{}_train.csv'.format(output_datadir,dt2,f), header=header)
        df = pd.DataFrame(data[f][unseen])
        df.columns = header
        df.to_csv('{}/{}_f{}_test.csv'.format(output_datadir,dt2,f), header=header)

pre = 'rand-sensors-'
mid = 'usa' if args.usa else 'delhi' if not canada else 'canada-days' if canada == 1 else 'canada-year' if canada == 2 else ''
post = '-st' if args.spatiotemporal else ''
save(S, pre + mid + post + '.bin')
print('Done')

Cuda: False | Device: cpu
Num Sensors: 60
Policy: random
Reading 2020-11-12_all.csv


<ipython-input-2-622e561bc805>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = dfHour.dateTime.dt.round('{}min'.format(grouptime))
<ipython-input-2-622e561bc805>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = pd.to_datetime(dfHour.dateTime)
<ipython-input-2-622e561bc805>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Reading 2020-11-13_all.csv


<ipython-input-2-622e561bc805>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = dfHour.dateTime.dt.round('{}min'.format(grouptime))
<ipython-input-2-622e561bc805>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = pd.to_datetime(dfHour.dateTime)
<ipython-input-2-622e561bc805>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Reading 2020-11-14_all.csv


...

Reading 2021-01-29_all.csv


<ipython-input-2-622e561bc805>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = dfHour.dateTime.dt.round('{}min'.format(grouptime))
<ipython-input-2-622e561bc805>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = pd.to_datetime(dfHour.dateTime)
<ipython-input-2-622e561bc805>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Reading 2021-01-30_all.csv


<ipython-input-2-622e561bc805>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = dfHour.dateTime.dt.round('{}min'.format(grouptime))
<ipython-input-2-622e561bc805>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHour.dateTime = pd.to_datetime(dfHour.dateTime)
<ipython-input-2-622e561bc805>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Data Shape (118310, 5)
Done


## ARIMA

In [3]:
import numpy as np
import pandas as pd
import datetime
import random
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
random.seed(10)

usa = 0
mon_str = ['2020-11','2020-12','2021-01']
mons = ['Nov', 'Dec', 'Jan']
days = [0, 30, 61]

test_start = 15
grouptime = 30
km = 1
history = 3
test_portion = 36

lat_range = {'min': 28.486547, 'max': 28.719998, 'diff': 0.00451}
long_range = {'min': 77.100136, 'max': 77.320763, 'diff': 0.005085}

def data_preparation(datafile):
    #Put the file location
    df = pd.read_csv(datafile)
    #type casting
    df.pm1_0 = df.pm1_0.astype(float)
    df.pm2_5 = df.pm2_5.astype(float)
    df.pm10 = df.pm10.astype(float)

    # Ensuring Delhi region and removing outliers from data
    df = df[(df.lat.astype(int) == 28) &(df.long.astype(int) == 77)]
    df = df[(df.pm1_0<=1500) & (df.pm2_5<=1500) & (df.pm10<=1500) & (df.pm1_0>=20) & (df.pm2_5>=30) & (df.pm10>=30)]

    # rounding @30min
    df.dateTime = pd.to_datetime(df.dateTime)
    df.dateTime = df.dateTime.dt.round('{}min'.format(grouptime))
    # use time as a feature as well
    df.dateTime = df.dateTime.dt.hour*60 + df.dateTime.dt.minute
    df.dateTime %= 1440

    df = df[(df.dateTime>=300) & (df.dateTime<=1350)]
    def round_val(val, min, off):
        val1 = ((val - min) / off).astype(int)
        val2 = (val1 * off) + min
        return val2
    df.lat = round_val(df.lat.astype(float), lat_range['min'], lat_range['diff'])
    df.long = round_val(df.long.astype(float), long_range['min'], long_range['diff'])
    df['lat_grid'] = df.apply(
        lambda row: int(((row.lat - lat_range['min']) / lat_range['diff']) + 0.1),
        axis=1)
    df['long_grid'] = df.apply(
        lambda row: int(((row.long - long_range['min']) / long_range['diff']) + 0.1),
        axis=1)

    df = df.drop(columns = ['uid', 'deviceId'])

    df = df.pivot_table(index=['lat_grid','long_grid'], columns='dateTime', aggfunc='mean')['pm2_5']
    return df

def data_preparation_usa(DF, datafile):
    #Put the file location
    df = DF[pd.DatetimeIndex(DF.dateTime)==datafile]
    #type casting
    df.pm2_5 = df.pm2_5.astype(float)
    df.pm10 = df.pm10.astype(float)

    # rounding @30min
    df.dateTime = pd.to_datetime(df.dateTime)
    df.dateTime = df.dateTime.dt.round('{}min'.format(grouptime))
    # use time as a feature as well
    df.dateTime = df.dateTime.dt.hour*60 + df.dateTime.dt.minute
    df.dateTime %= 1440

    def round_val(val, min, off):
        val1 = ((val - min) / off).astype(int)
        val2 = (val1 * off) + min
        return val2
    df.lat = round_val(df.lat.astype(float), lat_range['min'], lat_range['diff'])
    df.long = round_val(df.long.astype(float), long_range['min'], long_range['diff'])
    df['lat_grid'] = df.apply(
        lambda row: int(((row.lat - lat_range['min']) / lat_range['diff']) + 0.1),
        axis=1)
    df['long_grid'] = df.apply(
        lambda row: int(((row.long - long_range['min']) / long_range['diff']) + 0.1),
        axis=1)

    df = df.drop(columns = ['uid', 'deviceId'])
    
    df = df.pivot_table(index=['lat_grid','long_grid'], columns='dateTime', aggfunc='mean')['pm2_5']
    return df

test_rmse = []
test_days = []
df_shape = []
tm_data, tm_train, tm_infer, tm_eval = [],[],[],[]

def tmsecs(tm2, tm1):
    return int((tm2 - tm1).total_seconds() * 1000)


def proc(df, idx=0):
    tm_train.append(0)
    tm_infer.append(0)
    random.seed(10)
    train_size = test_portion * history
    actual_test, predicted_test = pd.DataFrame(), pd.DataFrame()
    for i, row in df.iterrows():
        df_row = row.to_frame().reset_index()
        df_row.columns = [['time', 'pm2_5']]
        tm1 = datetime.datetime.now()
        model = ARIMA(df_row['pm2_5'][:train_size], order=(3, 1, 1))
        model_fit = model.fit()
        tm2 = datetime.datetime.now()
        df_row['forecast'] = model_fit.predict(start=train_size, end=df.shape[1] + 1, dynamic=True)
        tm3 = datetime.datetime.now()
        tm_train[-1] += tmsecs(tm2, tm1)
        tm_infer[-1] += tmsecs(tm3, tm2)
        # df_row[['pm2_5','forecast']].plot(figsize=(12,8))
        actual_row = df_row['pm2_5'][train_size:]
        predicted_row = df_row['forecast'][train_size:]
        actual_test = pd.concat([actual_test, actual_row]) # .append(actual_row)
        predicted_test = pd.concat([predicted_test, predicted_row])
        # predicted_test = predicted_test.append(predicted_row)

    tmStart = datetime.datetime.now()
    actual_test = np.array(actual_test)
    predicted_test = np.array(predicted_test)
    rmse = sqrt(mean_squared_error(actual_test[actual_test > 0], predicted_test[actual_test > 0]))
    tm_eval.append(tmsecs(datetime.datetime.now(), tmStart))
    test_rmse.append(rmse)
    df_shape.append(df.shape)
    print(test_days[-1], rmse)
    if 1:
        out = pd.DataFrame()
        out['day'] = test_days[-1:]
        out['rmse'] = test_rmse[-1:]
        out['DF'] = df_shape[-1:]
        out['tmData'] = tm_data[-1:]
        out['tmTrain'] = tm_train[-1:]
        out['tmInfer'] = tm_infer[-1:]
        out['tmEval'] = tm_eval[-1:]
        out['idx'] = idx
        out.to_csv('arima_results_1.csv', mode='a', header=None)

def proc_delhi():
    for test_idx in range(test_start,30+31+30+1):
        start = test_idx - history
        df = None
        tmStart = datetime.datetime.now()
        for i in range(start, start + history + 1):
            mon_idx = 2 if i > days[2] else 1 if i > days[1] else 0
            datafile = '{}/{}-{:02d}_all.csv'.format(DATA_FOLDER, mon_str[mon_idx], i - days[mon_idx])
            print('data', datafile)
            df1 = data_preparation(datafile)
            df = pd.concat([df, df1], axis=1)

        df = df.fillna(0)
        test_days.append('{} {}'.format(i - days[mon_idx], mons[mon_idx]))
        # df = df.set_index(['lat_grid','long_grid'])

        tm_data.append(tmsecs(datetime.datetime.now(), tmStart))
        proc(df, start)


def proc_usa():
    from dates import dates_usa

    DF = pd.read_csv('city_pollution_data.csv')
    DF = DF[['Date', 'Time', 'latitude', 'longitude', 'pm25_median', 'pm10_median']]
    DF.columns = ['dateTime', 'Time', 'lat', 'long', 'pm2_5', 'pm10']
    global lat_range, long_range
    lat_range = {'min': 21.3, 'max': 47.6, 'diff': 0.005}
    long_range = {'min': -157.9, 'max': -71.0, 'diff': 0.005}

    off = 0 if len(sys.argv) < 2 else int(sys.argv[1])
    for test_idx in range(history+off, len(dates_usa)):
        start = test_idx - history
        df = None
        tmStart = datetime.datetime.now()
        for i in range(start, start + history + 1):
            datafile = dates_usa[i]
            print('data', datafile)
            df1 = data_preparation_usa(DF, datafile)
            df = pd.concat([df, df1], axis=1)

        df = df.fillna(0)
        test_days.append(dates_usa[test_idx])
        # df = df.set_index(['lat_grid','long_grid'])

        tm_data.append(tmsecs(datetime.datetime.now(), tmStart))
        proc(df, start)

if usa:
    proc_usa()
else:
    proc_delhi()

out = pd.DataFrame()
out['day'] = test_days
out['TestRMSE'] = test_rmse
out['DF'] = df_shape
out['tmData'] = tm_data
out['tmTrain'] = tm_train
out['tmInfer'] = tm_infer
out['tmEval'] = tm_eval
out.to_csv('arima_results.csv', mode='a')

print('Mean', out['rmse'].mean())

data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-12_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-13_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-14_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-15_all.csv
15 Nov 161.7865304164225
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-13_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-14_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-15_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-16_all.csv
16 Nov 62.29633395800709
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-14_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-15_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-16_all.csv
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-17_all.csv
17 Nov 56.388332193976815
data /kaggle/input/airdelhi-dataset/PM Datasets//2020-11-15_all.csv
data /kaggle/input/airdelhi-dataset/PM D

LinAlgError: LU decomposition error.

In [ ]:
res = pd.read_csv('./arima_results_1.csv')
res.head()

(res['161.78648372469675'].sum() + 161.78648372469675) / (len(res) + 1)

In [ ]:
single_df = pd.read_csv("/kaggle/input/airdelhi-dataset/PM Datasets/2020-11-01_all.csv")

In [ ]:
single_df.head()

## IDW, RF and XGB

In [ ]:
!pip install polire

In [ ]:
import datetime
import argparse
import pandas as pd
import numpy as np
from polire import IDW
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

T = [0,1][1]
mode_t = ['AB','AC','C','ABC'][1]
mode_p = ['D','CD'][0]

class Args2:
    mfile=None
    fold=-1
    canada=0
    algo='rf'
    train_days=3
    seed=10
    temporal_scaling=T
    spatiotemporal=1
    datadir='./'

args = Args2()

maxFolds = 5
if args.spatiotemporal < 0 and mode_t == 'AB' and mode_p == 'CD':
    # Forecasting, single fold is enough
    maxFolds = 1
if args.fold < 0:
    args.fold = maxFolds

algos = ['xgb','idw','rf','err']
algo = algos[-1]
if args.algo is not None:
    if args.algo not in algos:
        raise 'Invalid algo'
    algo = args.algo
canada = args.canada
data_dir = args.datadir
np.random.seed(args.seed)

def get_suffixes(mode):
    suffixes = []
    if 'C' in mode or 'A' in mode:
        suffixes.append('train')
    if 'D' in mode or 'B' in mode:
        suffixes.append('test')
    return suffixes
train_suffix = get_suffixes(mode_t)

model = IDW(exponent=3) if algo=='idw' else RandomForestRegressor(n_estimators = 100, random_state = 0) if algo=='rf' else GradientBoostingRegressor() if algo=='xgb' else None
assert model is not None

def rename_cols(data):
    data.rename(
        columns={'dateTime': 'time', 'lat': 'latitude', 'long': 'longitude', 'pm2_5': 'PM25_Concentration',
                 'pm10': 'PM10_Concentration'}, inplace=True)

def return_data_0(train_input, test_input, with_scaling):
    train_output = np.array(train_input['pm2_5'])
    train_input = train_input[['dateTime','lat','long']]
    test_output = np.array(test_input['pm2_5'])
    test_input = test_input[['dateTime','lat','long']]
    if with_scaling:
        scaler = MinMaxScaler().fit(train_input)
        if args.temporal_scaling:
            data = scaler.transform(pd.concat((train_input, test_input)))
            test_input = data[len(train_input):]
            train_input = data[:len(train_input)]
        else:
            train_input = scaler.transform(train_input)
            test_input = scaler.transform(test_input)
    return train_input,train_output,test_input,test_output

def return_data_time(fold,data,with_scaling):
    train_input = None
    if 'A' in mode_t or 'B' in mode_t:
        for idx,dt in enumerate(data[:-1]):
            for suffix in train_suffix:
                input = pd.read_csv(data_dir+'/'+dt+'_f'+str(fold)+'_'+suffix+'.csv')
                if args.temporal_scaling:
                    input.dateTime += idx * 24 * 60
                train_input = pd.concat((train_input, input))
    if 'C' in mode_t:
        input = pd.read_csv(data_dir + '/' + data[-1] + '_f' + str(fold) + '_train.csv')
        if args.temporal_scaling:
            input.dateTime += (len(data)-1) * 24 * 60
        train_input = pd.concat((train_input, input))

    test_input = pd.read_csv(data_dir+'/'+data[-1]+'_f'+str(fold)+'_test.csv')
    if 'C' in mode_p:
        input = pd.read_csv(data_dir + '/' + data[-1] + '_f' + str(fold) + '_train.csv')
        test_input = pd.concat((input, test_input))
    if args.temporal_scaling:
        test_input.dateTime += (len(data)-1) * 24 * 60

    return return_data_0(train_input, test_input, with_scaling)

def train_model(model,train_input,train_output):
    model.fit(train_input, train_output)

def train_loss(model,train_input,train_output):
    train_input_1 = train_input
    train_pred = model.predict(train_input_1)
    err = mean_squared_error(train_pred, train_output, squared=False)
    return err

def infer_model(model,test_input):
    test_pred = model.predict(test_input)
    return test_pred

def eval_model(test_output, test_pred):
    err = mean_squared_error(test_output, test_pred, squared=False)
    return err

us_data = None
def return_data_forecast(fold, data):
    assert mode_t == 'AB' and mode_p == 'CD'

    data_input = None
    for idx,dt in enumerate(data):
        df = None
        for suffix in train_suffix:
            input = pd.read_csv(data_dir+'/'+dt+'_f'+str(fold)+'_'+suffix+'.csv')
            df = pd.concat((df, input))
        df = df.pivot_table(index=['lat','long'], columns='dateTime', aggfunc='mean')['pm2_5']
        data_input = pd.concat((data_input, df), axis=1)
    df = data_input.fillna(0)
    return df

def process(fold, date):
    tmStart = datetime.datetime.now()
    return_data = return_data_time
    train_input,train_output,test_input,test_output = return_data(fold=fold,data=date,with_scaling=True)
    tmPre = datetime.datetime.now()
    train_model(model,train_input,train_output)
    tmTrain = datetime.datetime.now()
    test_pred = infer_model(model,test_input)
    tmInfer = datetime.datetime.now()
    rmseTrain = train_loss(model,train_input,train_output)
    tmEvalTrain = datetime.datetime.now()

    tmEvalTrain = (tmEvalTrain - tmInfer).total_seconds()
    tmInfer = (tmInfer - tmTrain).total_seconds()
    tmTrain = (tmTrain - tmPre).total_seconds()
    tmPre = (tmPre - tmStart).total_seconds()

    tmEval = datetime.datetime.now()
    rmse = eval_model(test_output, test_pred)
    tmEval = (datetime.datetime.now() - tmEval).total_seconds()

    print(*date, "Fold: {}, RMSE: {}".format(fold, rmse))

    return '{},{},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}'.format('|'.join(date),fold,rmseTrain,rmse,tmPre,tmTrain,tmEvalTrain,tmInfer,tmEval)


def xform_day(day):
    arr = [0, 30, 61]
    w = 0 if day <= 30 else 1 if day <= 61 else 2
    mon = ['2020-11-', '2020-12-', '2021-01-'][w]
    date = mon + '{:02d}'.format(day - arr[w])
    return date

folds = [i for i in range(maxFolds)] if args.fold >= maxFolds else [args.fold]
header = 'Date,Fold,TrainRMSE,TestRMSE,tmPre,tmTrain,tmTrnEval,tmInfer,tmEval'

def proc_file(canada=0):
    mfile = '{}_{}{}_{}{}_{}{}.metrics.csv'.format(algo, ['delhi','canada','canada','usa'][canada], canada if canada in [1,2] else '', mode_t, 'T' if args.temporal_scaling else '', mode_p, '_st' if args.spatiotemporal>0 else '_20' if args.spatiotemporal<0 else '') if args.mfile is None else args.mfile
    f = open(mfile, 'a')
    print(',,'.join([header for _ in range(len(folds))]), file=f)
    return f

def proc_canada(canada):
    f = proc_file(canada)
    if canada == 1:
        dates = ['2015-02-12', '2015-02-13', '2015-03-19', '2015-03-25', '2015-03-27', '2015-04-01', '2015-04-10', '2015-04-15',
         '2015-04-17', '2015-04-21', '2015-04-24', '2015-05-12',  '2015-05-14', '2015-05-20', '2015-05-22', '2015-09-10', '2015-09-23', '2015-11-23']
    else:
        dates = ['canada_20{:02d}'.format(i) for i in range(6,17)]
    for day in range(args.train_days, len(dates)):
        date = []
        for i in range(args.train_days,-1,-1):
            date.append(dates[day-i])
        metrics = []
        for fold in folds:
            metrics.append(process(fold, date))
        print(',,'.join(metrics), file=f)

def proc_delhi():
    print("Processing Delhi")
    f = proc_file()
    nTestStartDay = 15
    nTotalDays = 91
    total_metrics = []
    for day in range(nTestStartDay, nTotalDays+1):
        date = []
        for i in range(args.train_days,-1,-1):
            date.append(xform_day(day-i))
        metrics = []
        for fold in folds:
            metrics.append(process(fold, date))
        # print(',,'.join(metrics), file=f)
        total_metrics.append(metrics)
    return total_metrics

def proc_usa():
    f = proc_file(3)
    dates = dates_usa
    for day in range(args.train_days, len(dates)):
        date = []
        for i in range(args.train_days, -1, -1):
            date.append(dates[day - i])
        metrics = []
        for fold in folds:
            metrics.append(process(fold, date))
        print(',,'.join(metrics), file=f)

if canada < 0:
    proc_delhi()
    proc_canada(1)
    proc_canada(2)
    proc_usa()
elif canada == 0:
    out = proc_delhi()
elif canada == 1:
    proc_canada(canada)
else:
    proc_usa()

In [ ]:
rmse_scores = [float(out[i][j].split(',')[3])
              for i in range(len(out))
              for j in range(len(out[i]))
              ]
sum(rmse_scores) / len(rmse_scores)

## IDW

In [ ]:
algo = 'idw'
model = IDW(exponent=3) if algo=='idw' else RandomForestRegressor(n_estimators = 100, random_state = 0) if algo=='rf' else GradientBoostingRegressor() if algo=='xgb' else None
assert model is not None

out_idw = proc_delhi()

In [ ]:
rmse_scores_idw = [float(out_idw[i][j].split(',')[3])
              for i in range(len(out_idw))
              for j in range(len(out_idw[i]))
              ]
sum(rmse_scores_idw) / len(rmse_scores_idw)

## Gradient Boosted Tree

In [ ]:
algo = 'xgb'
model = IDW(exponent=3) if algo=='idw' else RandomForestRegressor(n_estimators = 100, random_state = 0) if algo=='rf' else GradientBoostingRegressor() if algo=='xgb' else None
assert model is not None

out_xgb = proc_delhi()

In [ ]:
rmse_scores_xgb = [float(out_xgb[i][j].split(',')[3])
              for i in range(len(out_xgb))
              for j in range(len(out_xgb[i]))
              ]
sum(rmse_scores_xgb) / len(rmse_scores_xgb)

## CatBoostRegressor

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(verbose=0)
out_cat = proc_delhi()

In [ ]:
rmse_scores_cat = [float(out_cat[i][j].split(',')[3])
              for i in range(len(out_cat))
              for j in range(len(out_cat[i]))
              ]
sum(rmse_scores_cat) / len(rmse_scores_cat)

## Random Forest Next Level

In [ ]:
model = RandomForestRegressor(n_estimators = 200, random_state = 0)
out_rf = proc_delhi()

In [ ]:
rmse_scores_rf = [float(out_rf[i][j].split(',')[3])
              for i in range(len(out_rf))
              for j in range(len(out_rf[i]))
              ]
sum(rmse_scores_rf) / len(rmse_scores_rf)

## Basic Ridge

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge()
out_ridge = proc_delhi()

In [ ]:
rmse_scores_ridge = [float(out_ridge[i][j].split(',')[3])
              for i in range(len(out_ridge))
              for j in range(len(out_ridge[i]))
              ]
sum(rmse_scores_ridge) / len(rmse_scores_ridge)

## XGBoost

In [ ]:
from xgboost import XGBRegressor

algo = 'xgb'
model = XGBRegressor(max_depth=6) # IDW(exponent=3) if algo=='idw' else RandomForestRegressor(n_estimators = 100, random_state = 0) if algo=='rf' else GradientBoostingRegressor() if algo=='xgb' else None
assert model is not None

out_xgb = proc_delhi()

In [ ]:
rmse_scores_xgb = [float(out_xgb[i][j].split(',')[3])
              for i in range(len(out_xgb))
              for j in range(len(out_xgb[i]))
              ]
sum(rmse_scores_xgb) / len(rmse_scores_xgb)